In [89]:
import openpyxl as opx

In [90]:
wb = opx.load_workbook('../sample/sample.xlsx')

In [91]:
wb.sheetnames

['Sheet1', 'Sheet2']

In [92]:
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for row in ws.iter_rows():
        for cell in row:
            print(cell.coordinate)
            if cell.comment is not None:
                print(cell.comment.text)

A1
jellydonuts:
%%begin eco
key: simple_int
parser: int
%%end eco
A2
A3
A4
A5
jellydonuts:

%%begin eco
key: first_name
type: str
param: {token:0}
%%end eco
%%begin eco
key: last_name
type: str
param: {token:1}
%%end eco

A6
jellydonuts:
%%begin eco
key: empty
parser: str
%%end eco
A1
jellydonuts:
%%begin eco
key: second_sheet_int
parser: int_with_offset
param: {offset: 1}
validators:
  - {name: more_than, threshold: 0} 
  - {name: less_than, theshold: 0} 
%%end eco


In [93]:
ws = wb['Sheet1']
comment = ws['A1'].comment

In [94]:
comment.text

'jellydonuts:\n%%begin eco\nkey: simple_int\nparser: int\n%%end eco'

In [95]:
from typing import List, Dict, Any
import logging
import yaml
from dataclasses import dataclass
logging.getLogger().setLevel(logging.DEBUG)

@dataclass
class ECOBlock:
    start_line: int
    end_line: int
    raw: str
    
    def parse(self) -> Any:
        return yaml.load(self.raw)
        


def parse_eco(lines, start_marker="%%begin eco", end_marker="%%end eco")->List[Dict[str, Any]]:
    in_marker = False
    current_str = ''
    start_line = None
    end_line = None
    ret = []
    for i, line in enumerate(lines.splitlines(keepends=True)):
        print(line)
        if line.strip() == start_marker:
            logging.debug(f'start {i}')
            if in_marker:
                raise ValueError(f"Expect end marker before another begin marker at line {i}")
            in_marker = True
            start_line = i
        elif line.strip() == end_marker:
            logging.debug(f'end {i}')
            if not in_marker:
                raise ValueError(f"Expect another begin marker at line {i}")
            in_marker = False
            end_line = i
            ret.append(ECOBlock(
                start_line = start_line,
                end_line = end_line,
                raw = current_str
            ))
            current_str = ''
        elif in_marker:
            logging.debug(f'in_marker: {i} {line!r}')
            current_str += line
    return ret

In [96]:
parse_eco(comment.text)

DEBUG:root:start 1
DEBUG:root:in_marker: 2 'key: simple_int\n'
DEBUG:root:in_marker: 3 'parser: int\n'
DEBUG:root:end 4


jellydonuts:

%%begin eco

key: simple_int

parser: int

%%end eco


[ECOBlock(start_line=1, end_line=4, raw='key: simple_int\nparser: int\n')]

In [97]:
ws = wb['Sheet1']
comment = ws['A5'].comment
parse_eco(comment.text)

DEBUG:root:start 2
DEBUG:root:in_marker: 3 'key: first_name\n'
DEBUG:root:in_marker: 4 'type: str\n'
DEBUG:root:in_marker: 5 'param: {token:0}\n'
DEBUG:root:end 6
DEBUG:root:start 7
DEBUG:root:in_marker: 8 'key: last_name\n'
DEBUG:root:in_marker: 9 'type: str\n'
DEBUG:root:in_marker: 10 'param: {token:1}\n'
DEBUG:root:end 11


jellydonuts:



%%begin eco

key: first_name

type: str

param: {token:0}

%%end eco

%%begin eco

key: last_name

type: str

param: {token:1}

%%end eco



[ECOBlock(start_line=2, end_line=6, raw='key: first_name\ntype: str\nparam: {token:0}\n'),
 ECOBlock(start_line=7, end_line=11, raw='key: last_name\ntype: str\nparam: {token:1}\n')]